In [ ]:
!pip3 install bnbphoneticparser
!pip3 install googletrans==3.1.0a0
!pip3 install lingua-language-detector
!apt update
!apt-get update
!apt-get install -y libenchant-2-dev
!pip install pyenchant
!pip3 install emoji

  Preparing metadata (setup.py) ... done
  Created wheel for bnbphoneticparser: filename=bnbphoneticparser-0.1.5-py3-none-any.whl size=9527 sha256=8c2cd08fc9f9c78fab6737dcd41c558c770863c95faf3323f97fce946628102e
  Stored in directory: /root/.cache/pip/wheels/54/5a/f9/c80ff9b79e166e1586941f28deb6579def90b2fad833db5800
Successfully built bnbphoneticparser
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 4.5 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3

In [ ]:
import googletrans
from bnbphoneticparser import BanglishToBengali
from bnbphoneticparser import BengaliToBanglish
import time
import string
import re
import enchant
from lingua import Language, LanguageDetectorBuilder
import emoji
import pandas as pd

In [ ]:
translator = googletrans.Translator()
banglish2bengali = BanglishToBengali()
bengali2banglish = BengaliToBanglish()
d = enchant.Dict("en_US")
languages = [Language.ENGLISH, Language.BENGALI]
detector = LanguageDetectorBuilder.from_languages(*languages).build()
special_characters = "!\"#$%&\'()*+,-./:;<=>?@[\]^_`{|}~।"

In [ ]:
def spaceAllocate(txt):
  txt = str(txt)
  text = txt.translate(str.maketrans({key: " {0} ".format(key) for key in special_characters}))
  return text

In [ ]:
def spliteKeyWord(str):
    regex = r"[\u0980-\u09FF]+|[0-9]+|[,.।]+|[a-zA-Z]+\'*[a-z]*"
    matches = re.findall(regex, str, re.UNICODE)
    return matches

In [ ]:
def processSplitting(lst):
  new_lst = []
  for wrd in lst:
    result = spliteKeyWord(wrd)
    if len(result) == 1:
      new_lst.append(result[0])
      continue
    else:
      # print("i was here")
      for i in range(len(result)):
        new_lst.append(result[i])
  return new_lst

In [ ]:
def clean_txt(txt):
  txt = txt.replace("\n", " ")
  txt = emoji.replace_emoji(txt)
  bn_txt = re.split(r'[` \=~!#@$%^&.(),\—\_\-\[\]{};:\'\\"|<<>?]''', txt)
  bn_txt = list(filter(lambda item: item.strip(), bn_txt))
  print(bn_txt)
  return bn_txt

In [ ]:
def BFRD_Translate(txt):
  trns_txt = ""

  txt = spaceAllocate(txt)
  s_txt = clean_txt(txt)
  # print(s_txt)
  bn_txt = processSplitting(s_txt)

  for element in bn_txt:
    if element == "।":
      trns_txt = trns_txt + " " + element
      continue
    elif d.check(element):
      #print(element)
      if element.isdigit():
        trns_txt = trns_txt + " " + translator.translate(element, src='bn', dest='bn').text
      else:
        trns_txt = trns_txt + " " + translator.translate(element, src='en', dest='bn').text
    else:
      if element.isdigit():
        trns_txt = trns_txt + " " + translator.translate(element, src='bn', dest='bn').text
      elif detector.detect_language_of(element) == Language.BENGALI:
        trns_txt = trns_txt + " " + element
        continue
      else:
        if detector.detect_language_of(element) == Language.ENGLISH:
          trns_txt = trns_txt + " " + translator.translate(element, src='en', dest='bn').text
        else:
          trns_txt = trns_txt + " " + banglish2bengali.parse(element)
    time.sleep(1)

  return trns_txt

In [ ]:
def Second_level_Translate(txt):
    trns_txt = ""
    txt = spaceAllocate(txt)
    txt = clean_txt(txt)
    cn_txt = processSplitting(txt)
    #print(cn_txt)
    for element in cn_txt:
        if element == "।":
          trns_txt = trns_txt + " " + element
          continue
        else:
            if detector.detect_language_of(element) == Language.ENGLISH:
               trns_txt = trns_txt + " " + banglish2bengali.parse(element)
            else:
                trns_txt = trns_txt + " " + element
                continue
    return trns_txt

In [ ]:
df = pd.read_csv("/content/newfake.csv")
df.head()

,Review,Label
0,যেতে যেতে পথে বসন্তের এই দিনে দেখা হয়ে যায় আব...,0
1,💝💝Color Cafe💝💝 - সত্যি অসাধারন!!\r\n ...,0
2,"রাজা রাজ্য দখল করবে এটাই সাভাবিক,\r\nকিন্তু বি...",0
3,রামেন😋\r\n\r\nI think It's a best ramen in tow...,0
4,Rashi is not allow at Taste Blast ™️.🙂\r\n🟥 49...,0


In [ ]:
# testing
# txt = "Hello, world! 😃\nThis is an example text, with some special chars: #$% &*."
# txt=spaceAllocate(txt)
# clean_txt(txt)
txt = "Hello, আমি বাংলায় কথা বলি। it was amazing. ami onek khushi"
BFRD_Translate(txt)

['Hello', 'আমি', 'বাংলায়', 'কথা', 'বলি', '।', 'it', 'was', 'amazing', 'ami', 'onek', 'khushi']


' হ্যালো আমি বাংলায় কথা বলি । এটা ছিল আশ্চর্যজনক আমি এক khushi'

In [ ]:
result_list = []

for txt in df.Review:
  x = BFRD_Translate(txt)
  y = Second_level_Translate(x)
  result_list.append(y)

['যেতে', 'যেতে', 'পথে', 'বসন্তের', 'এই', 'দিনে', 'দেখা', 'হয়ে', 'যায়', 'আবারও', 'My', 'Chef', 'এর', 'সাথে', '।', 'রাস্তার', 'পাশ', 'দিয়ে', 'হেটে', 'যাওয়ার', 'সময়', 'নজর', 'করার', 'মত', 'একটি', 'পরিবেশ', '।', 'বিশেষ', 'করে', 'তাদের', 'বাহিরের', 'আপনার', 'প্রি়জনদের', 'সাথে', 'সুন্দর', 'কিছু', 'মুহূর্তের', 'পিক', 'তোলার', 'জন্য', 'বেস্ট', '।', 'আর', 'তাদের', 'ভিতরের', 'ইন্টেরিওর', 'টা', 'বেশ', 'ইউনিক', 'আপনার', 'ফ্রেন্ড', 'ফ্যামিলি', 'ওভারঅল', 'সবার', 'সাথেই', 'পিক', 'তোলার', 'জন্য', 'বেস্ট', 'একটু', 'ডেকোরেশন', '।', 'আর', 'তাদের', 'রেস্টুরেন্ট', 'এর', 'বয়স', 'বেশিদিন', 'হই', 'নি', '।', 'এই', 'অল্পকয়েকদিনের', 'মধ্যেই', 'তাদের', 'এই', 'ইন্টেরিয়র', 'আর', 'ফুড', 'কোয়ালিটির', 'জন্য', 'বিশেষ', 'ভাবে', 'পরিচিত', 'লাভ', 'করে', 'ফেলেছে', '।', 'এবার', 'আসি', 'খাবার', 'এর', 'কথায়', '।', 'খাবার', 'এর', 'দিক', 'দিয়েও', 'তারা', 'কোনো', 'অংশে', 'কম', 'নয়', '।', 'তাদের', 'মেনু', 'থেকে', 'আমি', 'অর্ডার', 'করলাম', 'আমার', 'বাজেট', 'এর', 'মধ্যে', 'সেট', 'মেনু', 'F', '।', 'এর', 'মধ্যে', 'ছিলো', 'থা

In [ ]:
print(len(result_list))
new_df = pd.DataFrame()
new_df['Translated'] = result_list
new_df['Label']=0
new_df.to_csv('Translated_My.csv', encoding = 'utf-8-sig', index=False)

295


In [ ]:
new_df.to_csv('Translated_Fake.csv', encoding = 'utf-8-sig', index=False)

In [ ]:
df = pd.read_csv("/content/non-fake.csv")
df.head()

,Review,Label
0,অকা...অবশেষে এত রিভিউ দেখে খাইতে গেলাম #Popeye...,1
1,"***অকি! কুনো লঞ্চ,বাসের টিকিট ছাড়াই আইজ মুই বর...",1
2,Octopus Madness De Flame বনানীতে পেয়ে গেলাম মন...,1
3,অক্টোপাস ফ্রাই...দিল্লি দরবার...বসুন্ধরা সিটি ...,1
4,Autograph Live Music Cafe (বনশ্রী ) আপনি নারু...,1


In [ ]:
result_list1 = []

for txt in df.Review:
  x = BFRD_Translate(txt)
  y = Second_level_Translate(x)
  result_list1.append(y)

['অকা', 'অবশেষে', 'এত', 'রিভিউ', 'দেখে', 'খাইতে', 'গেলাম', 'Popeye', 's', 'এর', '৯৯', 'টাকা', 'দামের', 'পাস্তা', 'পিক', 'দেখে', 'মোটেও', 'বিভ্রান্ত', 'হবেন', 'না', 'এখানে', 'দুইজনের', 'জন্য', 'পাস্তা', 'আসলে', 'ওনাদের', 'স্পেশাল', 'মেন্যুতে', 'পাস্তার', 'জন্য', 'দুইটা', 'অপশন', '।', 'একটা', 'সিংগেল', 'আরেকটা', 'ফর', 'টু', 'পারসন', 'সিংগেল', 'টার', 'দাম', '৯৯', 'টাকা', 'আর', 'ফর', 'টু', 'পারসন', 'এর', 'টা', '১৮০', 'টাকা', 'সিংগেল', 'টায়', 'একজনকে', 'এক', 'বাটিতে', 'খাইতে', 'দিবে', 'আর', 'ডাবল', 'টায়', 'দুইজনকে', 'একবাটিতে', 'খাইতে', 'দেয়', 'তাই', 'ডাবল', 'টায়', 'যতইই', 'দেক', 'না', 'কেন', 'মন', 'ভরে', 'নাই', 'আর', 'সিংগেল', 'টায়', 'মনে', 'হইল', 'পরিমানে', 'কিঞ্চিত', 'বেশি', 'দেয়আমি', 'ডাবলটা', 'অর্ডার', 'করেছিলাম', 'আগেভাগে', 'এত', 'কথা', 'জানিয়ে', 'দিলাম', 'নাইলে', 'পরে', 'আমার', 'মত', 'আরেকটা', 'বাটি', 'চাইয়া', 'বগী', 'হইতে', 'হবেএবারে', 'নম্বরের', 'পালায়', 'আসি', 'দাম', '৯', '/', '১০স্বাদ', '৮', '/', '১০পরিমান', '৭', '/', '১০', 'দাম', 'হিসাবে', 'ঠিকাসসে', 'আরেকটু', 'দিলে', 'ভাল্লাগতো

In [ ]:
print(len(result_list1))
new_df = pd.DataFrame()
new_df['Translated'] = result_list1

NameError: name 'result_list1' is not defined

# after recovery

In [ ]:
import pandas as pd

file_path = '/content/merged_fake.csv'
df = pd.read_csv(file_path)
df.rename(columns={'Translated-fake': 'Translated'}, inplace=True)
df['Label'] = 0
df.to_csv(file_path, encoding='utf-8-sig', index=False)

In [ ]:
result_list1=[]

In [ ]:
import ast
import pandas as pd

# Initialize the result list
result = []

# File path where the lists are stored
file_path = 'data.txt'  # Replace with your actual file path

# Read the file and parse the lists
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        line = line.strip()  # Remove leading and trailing whitespace
        if line:  # Ensure the line is not empty
            try:
                # Print the line for debugging
                # print(f"Parsing line: {line}")

                # Convert string representation of list into an actual list
                parsed_list = ast.literal_eval(line)  # Safer than eval()
                # result.append(parsed_list)
                add=" "
                for wrd in parsed_list:
                  add=add+" "+wrd
                result.append(add)
            except SyntaxError as se:
                print(f"SyntaxError parsing line: {line}")
                print(f"Error: {se}")
            except Exception as e:
                print(f"Error parsing line: {line}")
                print(f"Error: {e}")

# Print the result to verify
print("Result list:", result)

# Create DataFrame and save to CSV
new_df = pd.DataFrame()
new_df['Translated'] = result
new_df['Label'] = 1
new_df.to_csv('Translated_Non-Fake.csv', encoding='utf-8-sig', index=False)


Error parsing line: ['অনেকদিন', 'পর', 'আবার', 'একটি', 'রিভিউ', 'দিতে', 'ইচ্ছে', 'হল', 'তাই', 'র', 'না', 'দিয়ে', 'পারলামনা', '।', 'যাই', 'হোক', 'পয়েন্ট', 'এ', 'আসি', '।', 'হ্যা', 'এটা', 'আর', 'কোথাও', 'নয়', 'দ', 'ডাইনিং', 'loung', 'ওয়ারী', 'শাখায়', '।', 'তবে', 'এবার', 'এর', 'খাবার', 'টা', 'ছিল', 'একদম', 'নতুন', 'গিয়েছিলাম', 'বরাবরের', 'মত', 'পিজ্জা', 'খেতে', 'কিন্তু', 'মেনু', 'চেঞ্জ', 'আর', 'নতুন', 'আইটেম', 'দেখা', 'গেল', 'তাই', 'অরডার', 'করে', 'ফেললাম', 'নতুন', 'আগমন', 'দ', 'Musketeers', 'প্লাটার', 'আরেক', 'টি', 'বিষয়ে', 'হল', 'এটা', '1', 'ঃ', '3', '৭', 'আইটেম', 'এর', 'এই', 'প্ল্যাটার', 'এ', 'থাকছে', 'আমেরিকান', 'ভাজা', 'ভাত', 'বসন্ত', 'রোলচিকেন', 'ক', 'লা', 'কিংসেচুয়ান', 'ভাজা', 'মুরগি', 'ক্যান্টনিজ', 'বিফচিকেন', 'চিংড়ি', 'কাজু', 'বাদাম', 'সালাদ', 'মধু', 'সরিষা', 'সবুজ', 'তরকারী', 'সস', 'সব', 'চেয়ে', 'মজাদার', 'ছিল', 'মুরগী', 'ক', 'la', 'রাজা', '1', '3', 'এই', 'প্ল্যাটার', 'এর', 'দাম', 'ছিল', '890', 'tktaste', '8', '5', '10', 'এ', 'দ', 'ডাইনিং', 'লাউঞ্জ', 'ওয়ারী', 'ছাদ']['অনেকদিন',